In [1]:
from time import sleep, time
from subprocess import run
from pathlib import Path
from PIL import ImageColor
import pandas as pd
from IPython.display import display, Markdown, Latex, HTML
from remote_execution import RemoteExecution

# Connect to unreal

In [2]:
remote_exec = RemoteExecution()
remote_exec.start()
sleep(0.5)
for node in remote_exec.remote_nodes:
    remote_exec.open_command_connection(node.get("node_id"))


In [14]:

sleep(1)
remote_exec.has_command_connection()


True

# Load ldraw

In [10]:
ldraw_path = Path(R'C:\Program Files\Studio 2.0\ldraw')
bender_exe = Path(R'C:\Program Files\Blender Foundation\Blender 3.1\blender.exe')
place_lego_part = Path('Content/Python/place_lego_part.py')
import_script = Path('blender_import_script.py')
import_fbx = Path('Content/Python/import_fbx.py')
place_lego_part = Path('Content/Python/place_lego_part.py')
import_materials = Path('import_materials.py')

In [5]:
colors = Path(R'C:\Program Files\Studio 2.0\data\StudioColorDefinition.txt')
df = pd.read_csv(colors, delimiter='	')

In [6]:
df = df[['LDraw Color Code', 'RGB value', 'Alpha', 'Ins_CMYK']]

In [8]:
copy_to_ue_tool = []
file_to_import = R'c:\Other\Lego\piaggio_ape_v1.ldr'
with open(file_to_import) as f:
    for line in f.readlines():
        if line[0] == '1':
            brick = line.split()  # color 1, file 14
            color = brick[1]
            vector = ','.join(brick[2:5])
            matrix = ','.join(brick[5:14])
            brick = brick[14].split('.')[0]
            color_data = df[df['LDraw Color Code'] == int(color)]
            try:
                color_values = ImageColor.getcolor(
                    color_data['RGB value'].iat[0], "RGB")
            except:
                print(f'skipped color {int(color)}')
            color_r = color_values[0]/255.0
            color_g = color_values[1]/255.0
            color_b = color_values[2]/255.0
            alpha = color_data['Alpha'].iat[0]
            
            brick_path = Path().joinpath('models', f'{brick}.fbx')
            result = remote_exec.run_command(f'{place_lego_part.resolve()} -b "{brick}" -r " {matrix}" -v " {vector}" -m {color}')
            result_material = remote_exec.run_command(f'{import_materials.resolve()} -r {color_r} -g {color_g} -b {color_b} -a {alpha} -n {color}')
            if result_material['success'] == False:
                print(brick, "can't make materials")
                break
            if result['success'] == False:
                print(brick, "can't place. importing...")
                blender_import = run(
                    [bender_exe,
                     '--background', '--python',
                     import_script.resolve(), '--', brick, ldraw_path.resolve()]
                )
                if blender_import.returncode != 0:
                    print('blender fail')
                    break
                result = remote_exec.run_command(f'{import_fbx.resolve()} -f {brick_path.resolve()}')
                if result['success'] == False:
                    print(brick, "can't import")
                    break
                result = remote_exec.run_command(f'{place_lego_part.resolve()} -b "{brick}" -r " {matrix}" -v " {vector}" -m {color}')
                if result['success'] == False:
                    print(brick, "can't place after import")
                    break
            copy_to_ue_tool.append(f'{place_lego_part.resolve()} -b "{brick}" -r " {matrix}" -v " {vector}" -m {color}')
with open('last_commands.txt', 'w') as f:
    f.write('\n\n'.join(copy_to_ue_tool))


43712 can't place. importing...
15208 can't place. importing...
14704 can't place. importing...
15456 can't place. importing...
3022 can't place. importing...
92946 can't place. importing...
3024 can't place. importing...
3068b can't place. importing...
54200 can't place. importing...
42023 can't place. importing...
72454 can't place. importing...
3004 can't place. importing...
3023 can't place. importing...
3710 can't place. importing...
3623 can't place. importing...
99207 can't place. importing...
14418 can't place. importing...
15573 can't place. importing...
60481 can't place. importing...
3040b can't place. importing...
2450 can't place. importing...
3009 can't place. importing...
22885 can't place. importing...
3795 can't place. importing...
3001 can't place. importing...
4032a can't place. importing...
26604 can't place. importing...
85984 can't place. importing...
29119 can't place. importing...
29120 can't place. importing...
14417 can't place. importing...
15068 can't place.

In [13]:
blender_import = run(
                    [bender_exe,
                     '--background', '--python',
                     import_script.resolve(), '--', '2420', ldraw_path.resolve()]
                )